In [ ]:
%pip install openAI
%pip install langchain
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install pydantic==1.10.8

In [11]:
import openai
from langchain import *
from pydantic import *
import os 
import pandas as pd 
with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1] #for some reason there's a new line, so we remove it



# Let's try LangChain

Literally just grabbed the example from the website and modified. still have no idea how this works but whatever (for now) 


## Putting everything together

In [12]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List



# Define your desired data structure.
class extracted_data(BaseModel):
    descriptions: list = Field(description="The descriptions for the shape. This should be a list of one or more strings.")
    
    





input =pd.read_csv("6p_test.csv")

parser = PydanticOutputParser(pydantic_object=extracted_data)

info_query_generic = "Here's a partial transcript of someone describing an image to someone else. Extract verbatim the parts of the text that describe of the image. \
Do not include fillers like 'it looks like' or 'sorta'.  \
  or conversational words that aren't descriptive like without hedging or filler. \n\n \
  As an example, if the transcript was 'It looks like a magician, and, uhh, I think he's got a rabbit.', the response would be ['a magician','he's got a rabbit'].\n\n \
     As an example, if the transcript was 'yes, sorry, big triangle is on the right, like an arm', the response would be ['big triangle is on the right, like an arm'].\n\n \
      Return just the description in list format. Here's the text:\n\n "
#model_name = "gpt-3.5-turbo"
model_name="gpt-4"
temperature = 0
model = ChatOpenAI(temperature=temperature)


def code_data_with_langchain(df, model, parser, info_query_generic): 
  langchain_prompt = PromptTemplate(
    template="{query}\n",
    input_variables=["query"]
  )
  system_prompt = "You are a competent researcher who answers user queries."

  res = []
  for index in range(2):
  #for index, _ in df.iterrows():
    print(index)
    info_query = info_query_generic + df["text"][index] + "\n\n"
    _input = langchain_prompt.format_prompt(query=info_query)
    #print(_input)
    full_prompt=[SystemMessage(content=system_prompt), HumanMessage(content=_input.to_string())]
    #print(full_prompt)
    output = model(full_prompt)
    #parsed_output = parser.parse(output)
    string_output=output.content
    print(string_output)
    res.append(string_output)
  return res


res = code_data_with_langchain(input, model, parser,  info_query_generic)

output=input
output['gpt_out']=res
#output.to_csv("6p_out.csv")





0
['arms extended', 'one leg forward', 'one leg backwards', 'both legs on ground', 'right leg slightly higher', 'arms are head level', 'pointed slightly up', 'back leg is thicc', 'front leg is a triangle']
1
['one left straight on the floor', 'back leg pointed back at a 90 degree angle', 'arms forward', 'head tilted back', 'two triangle arms', 'one arm below the other']


ValueError: Length of values (2) does not match length of index (216)